In [1]:
import pybaseball as pyb
from pybaseball import statcast, pitching_stats, playerid_lookup, statcast_pitcher, statcast_pitcher_pitch_arsenal, pitching_stats_bref, pitching_stats_range
import numpy as np
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import re
import unicodedata
from datetime import datetime
from itertools import groupby
from operator import itemgetter
from fuzzywuzzy import process

C:\Users\johns\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
hist_tj_df = pd.read_csv('hist_tj.csv', index_col=0)
hist_tj_df

,Name,Age,Year,Throws,IP,G,GS,CG,SHO,sDR,Career Start,Career End,Inactive Years,Surgery,TJ Surgery Date,Surgeon(s),Country,Level,Total_IP,TJ Surgery Year
240,steve blass,31,1973,1,88.2,23,18,1,0,1,1972,1973,[],0.0,NaN,NaN,NaN,NaN,337.4,NaN
285,eddie fisher,36,1973,1,117.2,32,16,2,0,9,1972,1973,[],0.0,NaN,NaN,NaN,NaN,330.6,NaN
385,milt pappas,34,1973,1,162.0,30,29,1,1,8,1972,1973,[],0.0,NaN,NaN,NaN,NaN,357.0,NaN
463,steve arlin,28,1974,1,107.2,27,22,2,0,9,1972,1974,[],0.0,NaN,NaN,NaN,NaN,537.2,NaN
589,ernie mcanally,27,1974,1,128.2,25,21,5,2,6,1972,1974,[],0.0,NaN,NaN,NaN,NaN,445.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15334,zack wheeler,33,2023,1,192.0,32,32,0,0,0,2013,2023,"[2015, 2016]",1.0,2015-03-25,Dr. David Altchek,United States,MLB,1377.5,2015.0
15339,trevor williams,31,2023,1,144.1,30,30,0,0,0,2016,2023,[],0.0,NaN,NaN,NaN,NaN,857.1,NaN
15346,alex wood,32,2023,0,97.2,29,12,0,0,1,2013,2023,[],1.0,2009-01-01,NaN,United States,HS,1214.0,2009.0
15348,brandon woodruff,30,2023,1,67.0,11,11,1,1,0,2017,2023,[],0.0,NaN,NaN,NaN,NaN,678.7,NaN


In [3]:
hist_tj_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1244 entries, 240 to 15350
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             1244 non-null   object 
 1   Age              1244 non-null   int64  
 2   Year             1244 non-null   int64  
 3   Throws           1244 non-null   int64  
 4   IP               1244 non-null   float64
 5   G                1244 non-null   int64  
 6   GS               1244 non-null   int64  
 7   CG               1244 non-null   int64  
 8   SHO              1244 non-null   int64  
 9   sDR              1244 non-null   int64  
 10  Career Start     1244 non-null   int64  
 11  Career End       1244 non-null   int64  
 12  Inactive Years   1244 non-null   object 
 13  Surgery          1244 non-null   float64
 14  TJ Surgery Date  268 non-null    object 
 15  Surgeon(s)       197 non-null    object 
 16  Country          268 non-null    object 
 17  Level           

In [4]:
pitcher_data_df = hist_tj_df
pitcher_data_df

,Name,Age,Year,Throws,IP,G,GS,CG,SHO,sDR,Career Start,Career End,Inactive Years,Surgery,TJ Surgery Date,Surgeon(s),Country,Level,Total_IP,TJ Surgery Year
240,steve blass,31,1973,1,88.2,23,18,1,0,1,1972,1973,[],0.0,NaN,NaN,NaN,NaN,337.4,NaN
285,eddie fisher,36,1973,1,117.2,32,16,2,0,9,1972,1973,[],0.0,NaN,NaN,NaN,NaN,330.6,NaN
385,milt pappas,34,1973,1,162.0,30,29,1,1,8,1972,1973,[],0.0,NaN,NaN,NaN,NaN,357.0,NaN
463,steve arlin,28,1974,1,107.2,27,22,2,0,9,1972,1974,[],0.0,NaN,NaN,NaN,NaN,537.2,NaN
589,ernie mcanally,27,1974,1,128.2,25,21,5,2,6,1972,1974,[],0.0,NaN,NaN,NaN,NaN,445.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15334,zack wheeler,33,2023,1,192.0,32,32,0,0,0,2013,2023,"[2015, 2016]",1.0,2015-03-25,Dr. David Altchek,United States,MLB,1377.5,2015.0
15339,trevor williams,31,2023,1,144.1,30,30,0,0,0,2016,2023,[],0.0,NaN,NaN,NaN,NaN,857.1,NaN
15346,alex wood,32,2023,0,97.2,29,12,0,0,1,2013,2023,[],1.0,2009-01-01,NaN,United States,HS,1214.0,2009.0
15348,brandon woodruff,30,2023,1,67.0,11,11,1,1,0,2017,2023,[],0.0,NaN,NaN,NaN,NaN,678.7,NaN


In [5]:
# Function to fetch key_mlbam
def fetch_key_mlbam(row):
    try:
        # Splitting the name into first and last name
        first_name, last_name = row['Name'].split(' ')[0], ' '.join(row['Name'].split(' ')[1:])
        # Fetching player ID
        player_id_df = playerid_lookup(last_name, first_name)
        # Assuming the first result is the correct one, adjust as necessary
        key_mlbam = player_id_df.iloc[0]['key_mlbam']
        return key_mlbam
    except Exception as e:
        print(f"Error fetching key_mlbam for {row['Name']}: {e}")
        return pd.NA

# Apply the function to each row and create a new column 'key_mlbam'
pitcher_data_df['key_mlbam'] = pitcher_data_df.apply(fetch_key_mlbam, axis=1)

# Display the updated DataFrame
pitcher_data_df.head()

Gathering player lookup table. This may take a moment.
Error fetching key_mlbam for blue moon odom: single positional indexer is out-of-bounds
Error fetching key_mlbam for jr richard: single positional indexer is out-of-bounds
Error fetching key_mlbam for silvio martinez: single positional indexer is out-of-bounds
Error fetching key_mlbam for willie hernandez: single positional indexer is out-of-bounds
Error fetching key_mlbam for john henry johnson: single positional indexer is out-of-bounds
Error fetching key_mlbam for alejandro pena: single positional indexer is out-of-bounds
Error fetching key_mlbam for joaquin andujar: single positional indexer is out-of-bounds
Error fetching key_mlbam for oil can boyd: single positional indexer is out-of-bounds
Error fetching key_mlbam for pascual perez: single positional indexer is out-of-bounds
Error fetching key_mlbam for jose deleon: single positional indexer is out-of-bounds
Error fetching key_mlbam for jose mesa: single positional indexer i

Error fetching key_mlbam for carlos hernandez: single positional indexer is out-of-bounds
Error fetching key_mlbam for pablo lopez: single positional indexer is out-of-bounds
Error fetching key_mlbam for jesus luzardo: single positional indexer is out-of-bounds
Error fetching key_mlbam for german marquez: single positional indexer is out-of-bounds
Error fetching key_mlbam for nick martinez: single positional indexer is out-of-bounds
Error fetching key_mlbam for martin perez: single positional indexer is out-of-bounds
Error fetching key_mlbam for jose quintana: single positional indexer is out-of-bounds
Error fetching key_mlbam for carlos rodon: single positional indexer is out-of-bounds
Error fetching key_mlbam for eduardo rodriguez: single positional indexer is out-of-bounds
Error fetching key_mlbam for hyun jin ryu: single positional indexer is out-of-bounds
Error fetching key_mlbam for jose suarez: single positional indexer is out-of-bounds
Error fetching key_mlbam for ranger suarez

,Name,Age,Year,Throws,IP,G,GS,CG,SHO,sDR,...,Career End,Inactive Years,Surgery,TJ Surgery Date,Surgeon(s),Country,Level,Total_IP,TJ Surgery Year,key_mlbam
240,steve blass,31,1973,1,88.2,23,18,1,0,1,...,1973,[],0.0,NaN,NaN,NaN,NaN,337.4,NaN,111086
285,eddie fisher,36,1973,1,117.2,32,16,2,0,9,...,1973,[],0.0,NaN,NaN,NaN,NaN,330.6,NaN,114166
385,milt pappas,34,1973,1,162.0,30,29,1,1,8,...,1973,[],0.0,NaN,NaN,NaN,NaN,357.0,NaN,120213
463,steve arlin,28,1974,1,107.2,27,22,2,0,9,...,1974,[],0.0,NaN,NaN,NaN,NaN,537.2,NaN,110317
589,ernie mcanally,27,1974,1,128.2,25,21,5,2,6,...,1974,[],0.0,NaN,NaN,NaN,NaN,445.2,NaN,118506


In [6]:
playerid_lookup('teheran', fuzzy=True)

No identically matched names found! Returning the 5 most similar names.


,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,sherman,joe,122138,sherj101,shermjo01,1011880,1915.0,1915.0
1,torian,lonnie,-1,,torialo01,-1,1920.0,1920.0
2,teherán,julio,527054,tehej001,teherju01,6797,2011.0,2023.0
3,stephens,john,-1,,stephfr01,-1,1921.0,1921.0
4,stephens,john,407856,stepj001,stephjo03,1569,2002.0,2002.0
5,stephens,john,-1,,stephfr01,-1,1921.0,1921.0
6,stephens,john,407856,stepj001,stephjo03,1569,2002.0,2002.0


In [ ]:
odom_key = '119935'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Blue Moon Odom", 'key_mlbam'] = odom_key

In [ ]:
richard_key = '121145'
pitcher_data_df.loc[pitcher_data_df['Name'] == "J. R. Richard", 'key_mlbam'] = richard_key

In [ ]:
oil_key = '111312'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Oil Can Boyd", 'key_mlbam'] = oil_key

José DeJesús

Can't find. Drop.

In [ ]:
seo_key = '150242'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Jae Weong Seo", 'key_mlbam'] = seo_key

In [ ]:
kuo_key = '425539'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Hong-Chih Kuo", 'key_mlbam'] = kuo_key

In [ ]:
park_key = '120221'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Chan Ho Park", 'key_mlbam'] = park_key

In [ ]:
djcarrasco_key = '425647'
pitcher_data_df.loc[pitcher_data_df['Name'] == "D. J. Carrasco", 'key_mlbam'] = djcarrasco_key

In [ ]:
burnett_key = '150359'
pitcher_data_df.loc[pitcher_data_df['Name'] == "A. J. Burnett", 'key_mlbam'] = burnett_key

In [ ]:
wilson_key = '450351'
pitcher_data_df.loc[pitcher_data_df['Name'] == "C. J. Wilson", 'key_mlbam'] = wilson_key

In [ ]:
niese_key = '477003'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Jon Niese", 'key_mlbam'] = niese_key

In [ ]:
dickey_key = '285079'
pitcher_data_df.loc[pitcher_data_df['Name'] == "R. A. Dickey", 'key_mlbam'] = dickey_key

In [ ]:
griffin_key = '456167'
pitcher_data_df.loc[pitcher_data_df['Name'] == "A. J. Griffin", 'key_mlbam'] = griffin_key

In [ ]:
alvarez_key = '571439'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Jose Alvarez", 'key_mlbam'] = alvarez_key

In [ ]:
happ_key = '457918'
pitcher_data_df.loc[pitcher_data_df['Name'] == "J.A. Happ", 'key_mlbam'] = happ_key

In [ ]:
santiago_key = '502327'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Hector Santiago", 'key_mlbam'] = santiago_key

In [ ]:
mccullers_key = '621121'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Lance McCullers Jr.", 'key_mlbam'] = mccullers_key

In [ ]:
mattboyd_key = '571510'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Matthew Boyd", 'key_mlbam'] = mattboyd_key

Nestor Cortes

Can't find. Drop.

In [ ]:
eduardo_key = '593958'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Eduardo Rodriguez", 'key_mlbam'] = eduardo_key

In [ ]:
ryu_key = '547943'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Hyun-jin Ryu", 'key_mlbam'] = ryu_key

In [ ]:
teheran_key = '527054'
pitcher_data_df.loc[pitcher_data_df['Name'] == "Julio Teheran", 'key_mlbam'] = teheran_key

In [ ]:
pitcher_data_df['key_mlbam'].isna().value_counts()

In [ ]:
pitcher_data_df = pitcher_data_df[pitcher_data_df['Name'] != 'Nestor Cortes']

In [ ]:
pitcher_data_df = pitcher_data_df[pitcher_data_df['Name'] != 'José DeJesús']

In [1]:
# Example list of names
names_list = ['blue moon odom', 'jr richard', 'silvio martinez', 'willie hernandez', 
'john henry johnson', 'alejandro pena', 'joaquin andujar', 'oil can boyd', 
'pascual perez', 'jose deleon', 'jose mesa', 'jose de jesus', 
'jose guzman', 'jose bautista', 'angel miranda', 'dennis martinez', 
'francisco cordova', 'juan guzman', 'carlos perez', 'hipolito pichardo', 
'jose rosado', 'jose silva', 'osvaldo fernandez', 'ramon martinez', 
'jose mercedes', 'vladimir nunez', 'jose rijo', 'ruben quevedo',
'salomon torres', 'jesus sanchez', 'wilson alvarez', 'joaquin benoit',
'geremi gonzalez', 'sunwoo kim', 'jose lima', 'gustavo chacin', 
'orlando hernandez', 'byunghyun kim', 'victor santos', 'jae weong seo',
'julian tavarez', 'victor zambrano', 'shawn chacon', 'runelvys hernandez', 
'hungchih kuo', 'odalis perez', 'jose contreras', 'chan ho park',
'horacio ramirez', 'oliver perez', 'ryan rowlandsmith', 'dj carrasco', 
'livan hernandez', 'rodrigo lopez', 'joel pineiro', 'jojo reyes', 'javier vazquez',
'freddy garcia', 'ramon ortiz', 'jonathan sanchez', 'chienming wang', 
'erik bedard', 'aj burnett', 'felix doubront', 'wandy rodriguez', 
'cj wilson', 'jose fernandez', 'roberto hernandez', 'jon niese', 
'vidal nuno iii', 'alfredo simon', 'henderson alvarez iii', 'ra dickey',
'aj griffin', 'ubaldo jimenez', 'weiyin chen', 'bartolo colon', 'roenis elias',
'jaime garcia', 'miguel gonzalez', 'felix hernandez', 'hector noesi', 
'edinson volquez', 'ivan nova', 'jose alvarez', 'jhoulys chacin', 'ja happ', 
'jorge lopez', 'carlos martinez', 'hector santiago', 'reynaldo lopez', 
'lance mccullers jr', 'anibal sanchez', 'sandy alcantara', 'jaime barria', 'jose berrios', 
'matthew boyd', 'nestor cortes', 'domingo german', 'carlos hernandez', 'pablo lopez', 
'jesus luzardo', 'german marquez', 'nick martinez', 'martin perez', 'jose quintana',
'carlos rodon', 'eduardo rodriguez', 'hyun jin ryu', 'jose suarez', 'ranger suarez', 
'julio teheran', 'jose urena', 'julio urias', 'jose urquidy']

# Target name you are trying to match
all_player_names = ['Blue Moon Odom', 'J. R. Richard', 'Silvio Martínez',
                    'Willie Hernández', 'John Henry Johnson', 'Alejandro Peña', 
                    'Joaquín Andújar', 'Oil Can Boyd', 'Pascual Pérez', 'José DeLeón', 
                    'José Mesa', 'José DeJesús', 'José Guzmán', 'José Bautista',
                    'Ángel Miranda', 'Dennis Martínez', 'Francisco Córdova', 
                    'Juan Guzmán', 'Carlos Pérez', 'Hipólito Pichardo', 'José Rosado',
                    'José Silva', 'Osvaldo Fernández', 'Ramón Martínez', 'José Mercedes', 
                    'Vladimir Núñez', 'José Rijo', 'Rubén Quevedo', 'Salomón Torres',
                    'Jesús Sánchez', 'Wilson Álvarez', 'Joaquín Benoit', 'Geremi González',
                    'Sun-woo Kim', 'José Lima', 'Gustavo Chacín', 'Orlando Hernández', 
                    'Byung-hyun Kim', 'Víctor Santos', 'Jae Weong Seo', 
                    'Julián Tavárez', 'Víctor Zambrano', 'Shawn Chacón',
                    'Runelvys Hernández', 'Hong-Chih Kuo', 'Odalis Pérez',
                   'José Contreras', 'Chan Ho Park', 'Horacio Ramírez', 'Óliver Pérez', 
                    'Ryan Rowland-Smith', 'D. J. Carrasco', 'Liván Hernández', 
                    'Rodrigo López', 'Joel Piñeiro', 'Jo-Jo Reyes', 'Javier Vázquez', 
                    'Freddy García', 'Ramón Ortiz', 'Jonathan Sánchez', 'Chien-Ming Wang', 
                    'Érik Bédard', 'A. J. Burnett', 'Félix Doubront', 'Wandy Rodríguez', 
                    'C. J. Wilson', 'José Fernández', 'Roberto Hernández', 
                    'Jon Niese', 'Vidal Nuño', 'Alfredo Simón', 'Henderson Álvarez', 
                    'R. A. Dickey', 'A. J. Griffin', 'Ubaldo Jiménez', 'Wei-Yin Chen', 
                    'Bartolo Colón', 'Roenis Elías', 'Jaime García', 'Miguel González', 
                    'Félix Hernández', 'Héctor Noesí', 'Edinson Vólquez', 'Iván Nova', 
                   'José Álvarez', 'Jhoulys Chacín', 'J.A. Happ', 'Jorge López', 
                    'Carlos Martínez', 'Hector Santiago', 'Reynaldo López',
                    'Lance McCullers Jr.', 'Aníbal Sánchez', 'Sandy Alcántara', 'Jaime Barría', 
                    'José Berríos', 'Matthew Boyd', 'Nestor Cortes', 'Domingo Germán', 
                    'Carlos Hernández', 'Pablo López', 'Jesús Luzardo', 
                   'Germán Márquez', 'Martín Pérez', 'Carlos Rodón', 'José Urquidy', 
                    'Hong-Chih Kuo', 'Edinson Vólquez', 'Jose Alvarez']

# Function to find the best match for each name in names_list
def find_best_matches(names_list, all_player_names):
    best_matches = {}
    for name in names_list:
        match = process.extractOne(name, all_player_names)
        best_matches[name] = match
    return best_matches

# Get best matches
best_matches = find_best_matches(names_list, all_player_names)

# Print best matches
for name, match in best_matches.items():
    print(f"Original: {name}, Best Match: {match[0]}, Score: {match[1]}")

NameError: name 'process' is not defined

In [ ]:
updated_names = {
    "blue moon odom": "Blue Moon Odom",
    "jr richard": "J. R. Richard",
    "silvio martinez": "Silvio Martínez",
    "willie hernandez": "Willie Hernández",
    "john henry johnson": "John Henry Johnson",
    "alejandro pena": "Alejandro Peña",
    "joaquin andujar": "Joaquín Andújar",
    "oil can boyd": "Oil Can Boyd",
    "pascual perez": "Pascual Pérez",
    "jose deleon": "José DeLeón",
    "jose mesa": "José Mesa",
    "jose de jesus": "José DeJesús",
    "jose guzman": "José Guzmán",
    "jose bautista": "José Bautista",
    "angel miranda": "Ángel Miranda",
    "dennis martinez": "Dennis Martínez",
    "francisco cordova": "Francisco Córdova",
    "juan guzman": "Juan Guzmán",
    "carlos perez": "Carlos Pérez",
    "hipolito pichardo": "Hipólito Pichardo",
    "jose rosado": "José Rosado",
    "jose silva": "José Silva",
    "osvaldo fernandez": "Osvaldo Fernández",
    "ramon martinez": "Ramón Martínez",
    "jose mercedes": "José Mercedes",
    "vladimir nunez": "Vladimir Núñez",
    "jose rijo": "José Rijo",
    "ruben quevedo": "Rubén Quevedo",
    "salomon torres": "Salomón Torres",
    "jesus sanchez": "Jesús Sánchez",
    "wilson alvarez": "Wilson Álvarez",
    "joaquin benoit": "Joaquín Benoit",
    "geremi gonzalez": "Geremi González",
    "sunwoo kim": "Sun-woo Kim",
    "jose lima": "José Lima",
    "gustavo chacin": "Gustavo Chacín",
    "orlando hernandez": "Orlando Hernández",
    "byunghyun kim": "Byung-hyun Kim",
    "victor santos": "Víctor Santos",
    "jae weong seo": "Jae Weong Seo",
    "julian tavarez": "Julián Tavárez",
    "victor zambrano": "Víctor Zambrano",
    "shawn chacon": "Shawn Chacón",
    "runelvys hernandez": "Runelvys Hernández",
    "hungchih kuo": "Hong-Chih Kuo",
    "odalis perez": "Odalis Pérez",
    "jose contreras": "José Contreras",
    "chan ho park": "Chan Ho Park",
    "horacio ramirez": "Horacio Ramírez",
    "oliver perez": "Óliver Pérez",
    "ryan rowlandsmith": "Ryan Rowland-Smith",
    "dj carrasco": "D. J. Carrasco", 
    "livan hernandez": "Liván Hernández", 
    "rodrigo lopez": "Rodrigo López",
    "joel pineiro": "Joel Piñeiro",
    "jojo reyes": "Jo-Jo Reyes",
    "javier vazquez": "Javier Vázquez", 
    "freddy garcia": "Freddy García", 
    "ramon ortiz": "Ramón Ortiz",
    "jonathan sanchez": "Jonathan Sánchez", 
    "chienming wang": "Chien-Ming Wang", 
    "erik bedard": "Érik Bédard", 
    "aj burnett": "A. J. Burnett", 
    "felix doubront": "Félix Doubront",
    "wandy rodriguez": "Wandy Rodríguez", 
    "cj wilson": "C. J. Wilson", 
    "jose fernandez": "José Fernández",
    "roberto hernandez": "Roberto Hernández",
    "jon niese": "Jon Niese",
    "vidal nuno iii": "Vidal Nuño", 
    "alfredo simon": "Alfredo Simón",
    "henderson alvarez iii": "Henderson Álvarez",
    "ra dickey": "R. A. Dickey", 
    "aj griffin": "A. J. Griffin", 
    "ubaldo jimenez": "Ubaldo Jiménez", 
    "weiyin chen": "Wei-Yin Chen",
    "bartolo colon": "Bartolo Colón",
    "roenis elias": "Roenis Elías",
    "jaime garcia": "Jaime García",
    "miguel gonzalez": "Miguel González",
    "felix hernandez": "Félix Hernández", 
    "hector noesi": "Héctor Noesí", 
    "edinson volquez": "Edinson Vólquez", 
    "ivan nova": "Iván Nova",
    "jose alvarez": "Jose Alvarez",
    "jhoulys chacin": "Jhoulys Chacín",
    "ja happ": "J.A. Happ",
    "jorge lopez": "Jorge López",
    "carlos martinez": "Carlos Martínez",
    "hector santiago": "Hector Santiago",
    "reynaldo lopez": "Reynaldo López",
    "lance mccullers jr": "Lance McCullers Jr.",
    "anibal sanchez": "Aníbal Sánchez",
    "sandy alcantara": "Sandy Alcántara",
    "jaime barria": "Jaime Barría",
    "jose berrios": "José Berríos",
    "matthew boyd": "Matthew Boyd",
    "nestor cortes": "Nestor Cortes",
    "domingo german": "Domingo Germán",
    "carlos hernandez": "Carlos Hernández",
    "pablo lopez": "Pablo López",
    "jesus luzardo": "Jesús Luzardo",
    "german marquez": "Germán Márquez",
    "nick martinez": "Dennis Martínez",
    "martin perez": "Martín Pérez", 
    "jose quintana": "José Quintana",
    "carlos rodon": "Carlos Rodón",
    "eduardo rodriguez": "Eduardo Rodriguez", 
    "hyun jin ryu": "Hyun-jin Ryu",
    "jose suarez": "José Suarez",
    "ranger suarez": "Ranger Suárez",
    "julio teheran": "Julio Teheran",
    "jose urena": "José Ureña",
    "julio urias": "Julio Urías", 
    "jose urquidy": "José Urquidy",
}

pitcher_data_df['Name'] = pitcher_data_df['Name'].map(updated_names).fillna(pitcher_data_df['Name'])
pitcher_data_df

In [ ]:
'blue moon odom', 'jr richard', 'silvio martinez', 'willie hernandez', 'john henry johnson',
'alejandro pena', 'joaquin andujar', 'oil can boyd', 'pascual perez',
'jose deleon', 'jose mesa', 'jose de jesus', 'jose guzman', 'jose bautista',
'angel miranda', 'dennis martinez', 'francisco cordova', 'juan guzman', 'carlos perez',
'hipolito pichardo', 'jose rosado', 'jose silva', 'osvaldo fernandez', 
'ramon martinez', 'jose mercedes', 'vladimir nunez', 'jose rijo', 'ruben quevedo',
'salomon torres', 'jesus sanchez', 'wilson alvarez', 'joaquin benoit',
'geremi gonzalez', 'sunwoo kim', 'jose lima', 'gustavo chacin', 
'orlando hernandez', 'byunghyun kim', 'victor santos', 'jae weong seo',
'julian tavarez', 'victor zambrano', 'shawn chacon', 'runelvys hernandez', 
'hungchih kuo', 'odalis perez', 'jose contreras', 'chan ho park',
'horacio ramirez', 'oliver perez', 'ryan rowlandsmith', 'dj carrasco', 
'livan hernandez', 'rodrigo lopez', 'joel pineiro', 'jojo reyes', 'javier vazquez',
'freddy garcia', 'ramon ortiz', 'jonathan sanchez', 'chienming wang', 
'erik bedard', 'aj burnett', 'felix doubront', 'wandy rodriguez', 
'cj wilson', 'jose fernandez', 'roberto hernandez', 'jon niese', 
'vidal nuno iii', 'alfredo simon', 'henderson alvarez iii', 'ra dickey',
'aj griffin', 'ubaldo jimenez', 'weiyin chen', 'bartolo colon', 'roenis elias',
'jaime garcia', 'miguel gonzalez', 'felix hernandez', 'hector noesi', 
'edinson volquez', 'ivan nova', 'jose alvarez', 'jhoulys chacin', 'ja happ', 
'jorge lopez', 'carlos martinez', 'hector santiago', 'reynaldo lopez', 
'lance mccullers jr', 'anibal sanchez', 'sandy alcantara', 'jaime barria', 'jose berrios', 
'matthew boyd', 'nestor cortes', 'domingo german', 'carlos hernandez', 'pablo lopez', 
'jesus luzardo', 'german marquez', 'nick martinez', 'martin perez', 'jose quintana',
'carlos rodon', 'eduardo rodriguez', 'hyun jin ryu', 'jose suarez', 'ranger suarez', 
'julio teheran', 'jose urena', 'julio urias', 'jose urquidy'

In [269]:
pitcher_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1242 entries, 240 to 15350
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             1242 non-null   object 
 1   Age              1242 non-null   int64  
 2   Year             1242 non-null   int64  
 3   Throws           1242 non-null   int64  
 4   IP               1242 non-null   float64
 5   G                1242 non-null   int64  
 6   GS               1242 non-null   int64  
 7   CG               1242 non-null   int64  
 8   SHO              1242 non-null   int64  
 9   sDR              1242 non-null   int64  
 10  Career Start     1242 non-null   int64  
 11  Career End       1242 non-null   int64  
 12  Inactive Years   1242 non-null   object 
 13  Surgery          1242 non-null   float64
 14  TJ Surgery Date  268 non-null    object 
 15  Surgeon(s)       197 non-null    object 
 16  Country          268 non-null    object 
 17  Level           

In [ ]:
pitcher_data_df.to_csv('pitcher_key_df.csv')

In [ ]:
player_id = pyb.playerid_lookup('ohtani', 'shohei')

In [ ]:
playerid_lookup('ohtani', 'shohei')

In [ ]:
ohtani_stats = statcast_pitcher('2018-01-01', '2023-10-01', 660271)
ohtani_stats

In [ ]:
ohtani_stats.info()

In [ ]:
ohtani_stats['game_date'].value_counts()

In [ ]:
ohtani_stats['game_date'].unique

In [ ]:
ohtani_start = ohtani_stats[ohtani_stats['game_date'] == '2021-09-03']
ohtani_start

In [ ]:
ohtani_start.columns

In [ ]:
ohtani_start.info()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

Need to double check for pre/post season

In [ ]:
"""
all_player_stats = []

season_dates = {
    2008: ('2008-03-25', '2008-09-30'),
    2009: ('2009-04-05', '2009-10-06'),
    2010: ('2010-04-04', '2010-10-03'),
    2011: ('2011-03-31', '2011-09-28'),
    2012: ('2012-03-28', '2012-10-03'),
    2013: ('2013-03-31', '2013-09-30'),
    2014: ('2014-03-22', '2014-09-28'),
    2015: ('2015-04-05', '2015-10-04'),
    2016: ('2016-04-03', '2016-10-02'),
    2017: ('2017-04-02', '2017-10-01'),
    2018: ('2018-03-29', '2018-10-01'),
    2019: ('2019-03-20', '2019-09-29'),
    2020: ('2020-07-23', '2020-09-27'),
    2021: ('2021-04-01', '2021-10-03'),
    2022: ('2022-04-07', '2022-10-05'),
    2023: ('2023-03-30', '2023-10-01')
    
}

# Loop through each MLBAM ID in the surgery_df
for key_mlbam in surgery_df['key_mlbam'].unique():
    for year, dates in season_dates.items():
        start_date, end_date = dates
        # Fetch pitching stats from statcast
        player_stats = statcast_pitcher(start_date, end_date, key_mlbam)
        # Append the fetched stats to the list if player_stats is not None
        if player_stats is not None:
            all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_08_23_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_08_23_stats_df.head()
"""

In [244]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2018-03-29', '2018-10-01', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2018_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2018_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\3298240489.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2018_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2018-09-29,93.3,2.20,5.41,"Pérez, Oliver",641531,424144,field_out,hit_into_play,NaN,NaN,NaN,NaN,4.0,Hunter Dozier flies out to right fielder Melky...,R,R,L,KC,CLE,X,9.0,fly_ball,0,1,2018,0.91,1.26,-0.31,2.83,650619.0,NaN,460086.0,2,7,Bot,214.59,114.81,NaN,NaN,547379,NaN,NaN,-8.330570,-135.489359,-3.537231,12.754514,28.834540,-16.456552,3.55,1.63,288.0,84.5,38.0,92.4,2177.0,5.8,531811,424144,547379,475174,608070,518626,489267,488681,656185,466320,54.68,0.013,0.019,0.0,1.0,0.0,0.0,3.0,69,2,4-Seam Fastball,9,3,9,3,3,9,9,3,Standard,Standard,144.0,-0.001,-0.407
1,SI,2018-09-29,92.6,2.50,5.22,"Pérez, Oliver",641531,424144,NaN,called_strike,NaN,NaN,NaN,NaN,1.0,Hunter Dozier flies out to right fielder Melky...,R,R,L,KC,CLE,S,NaN,NaN,0,0,2018,1.48,0.28,-0.37,2.98,650619.0,NaN,460086.0,2,7,Bot,NaN,NaN,NaN,NaN,547379,NaN,NaN,-10.494050,-134.458927,-0.530389,20.104869,28.268887,-28.840385,3.49,1.70,NaN,NaN,NaN,92.3,1941.0,6.2,531811,424144,547379,475174,608070,518626,489267,488681,656185,466320,54.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,1,Sinker,9,3,9,3,3,9,9,3,Standard,Standard,101.0,0.000,-0.069
2,SL,2018-09-29,79.2,2.72,4.81,"Pérez, Oliver",460086,424144,single,hit_into_play,NaN,NaN,NaN,NaN,4.0,Alex Gordon singles on a ground ball to right ...,R,L,L,KC,CLE,X,9.0,ground_ball,0,1,2018,-0.78,-0.20,-0.44,2.14,NaN,444876.0,650619.0,2,7,Bot,158.58,86.19,NaN,NaN,547379,NaN,NaN,-5.409580,-115.029257,1.272910,-5.716481,20.118242,-34.375437,3.41,1.56,89.0,99.9,4.0,77.8,2197.0,5.3,531811,424144,547379,475174,608070,518626,489267,488681,656185,466320,55.22,0.533,0.480,0.9,1.0,1.0,0.0,4.0,68,2,Slider,8,3,8,3,3,9,9,3,Strategic,Standard,284.0,0.007,0.896
3,SI,2018-09-29,93.0,2.53,5.22,"Pérez, Oliver",460086,424144,NaN,called_strike,NaN,NaN,NaN,NaN,6.0,Alex Gordon singles on a ground ball to right ...,R,L,L,KC,CLE,S,NaN,NaN,0,0,2018,1.37,0.47,0.42,2.80,NaN,444876.0,650619.0,2,7,Bot,NaN,NaN,NaN,NaN,547379,NaN,NaN,-8.334664,-135.149771,-1.445371,18.395080,28.551413,-26.370682,3.41,1.62,NaN,NaN,NaN,92.7,2095.0,6.1,531811,424144,547379,475174,608070,518626,489267,488681,656185,466320,54.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,1,Sinker,8,3,8,3,3,8,8,3,Strategic,Standard,109.0,0.000,-0.060
4,SL,2018-09-29,77.9,2.55,5.01,"Pérez, Oliver",609275,424144,field_out,hit_into_play,NaN,NaN,NaN,NaN,11.0,Adalberto Mondesi pops out to catcher Roberto ...,R,R,L,KC,CLE,X,2.0,popup,1,0,2018,-0.76,-0.23,-0.84,2.82,NaN,444876.0,650619.0,1,7,Bot,112.86,208.61,NaN,NaN,547379,NaN,NaN,-5.795064,-113.095659,2.538368,-5.319335,19.644658,-34.813534,3.41,1.56,NaN,80.0,69.0,76.3,2186.0,5.2,531811,424144,547379,475174,608070,518626,489267,488681,656185,466320,55.32,0.000,0.000,0.0,1.0,0.0,0.0,3.0,67,2,Slider,8,3,8,3,3,8,8,3,Standard,Standard,287.0,-0.002,-0.553


In [245]:
all_2018_stats_df.to_csv('all_2018_stats_df.csv')

In [246]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2017-04-02', '2017-10-01', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2017_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2017_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\710815170.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2017_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2017-09-07,94.8,-2.68,6.27,"Benoit, Joaquín",592178,276542,field_out,hit_into_play,NaN,NaN,NaN,NaN,2.0,"Kris Bryant grounds out, third baseman Sean Ro...",R,R,R,PIT,CHC,X,5.0,ground_ball,0,0,2017,-1.01,1.41,0.11,3.09,NaN,664023.0,NaN,2,8,Top,106.60,163.24,NaN,NaN,553869,NaN,NaN,9.392586,-137.233135,-5.613659,-14.702656,31.677900,-13.541176,3.31,1.48,30.0,97.8,-4.0,93.6,2192.0,6.0,492193,276542,553869,605137,621559,446481,474568,607471,457705,656669,54.53,0.353,0.318,0.00,1.0,0.0,0.0,2.0,66,1,4-Seam Fastball,1,8,8,1,8,1,8,1,Infield shift,Standard,218.0,0.001,-0.325
1,CH,2017-09-07,85.1,-2.84,6.10,"Benoit, Joaquín",664023,276542,double,hit_into_play,NaN,NaN,NaN,NaN,11.0,Ian Happ doubles (16) on a fly ball to left fi...,R,L,R,PIT,CHC,X,7.0,fly_ball,2,2,2017,-1.45,0.62,-1.46,2.87,NaN,NaN,NaN,2,8,Top,62.02,138.28,NaN,NaN,553869,NaN,NaN,6.034954,-123.660884,-2.543159,-15.854333,21.528776,-25.575178,3.60,1.38,208.0,67.3,38.0,85.0,1340.0,6.0,492193,276542,553869,605137,621559,446481,474568,607471,457705,656669,54.50,0.627,0.606,1.25,1.0,1.0,1.0,4.0,65,5,Changeup,1,8,8,1,8,1,8,1,Standard,Standard,251.0,-0.001,0.244
2,FF,2017-09-07,94.7,-2.77,6.31,"Benoit, Joaquín",664023,276542,NaN,foul,NaN,NaN,NaN,NaN,1.0,Ian Happ doubles (16) on a fly ball to left fi...,R,L,R,PIT,CHC,S,NaN,NaN,2,1,2017,-1.22,1.48,-0.47,3.44,NaN,NaN,NaN,2,8,Top,NaN,NaN,NaN,NaN,553869,NaN,NaN,8.614072,-137.289711,-4.996384,-17.240459,30.795659,-12.763479,3.60,1.38,192.0,78.4,26.0,94.0,2136.0,6.1,492193,276542,553869,605137,621559,446481,474568,607471,457705,656669,54.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,4,4-Seam Fastball,1,8,8,1,8,1,8,1,Standard,Standard,222.0,0.000,-0.033
3,CH,2017-09-07,85.1,-2.88,5.96,"Benoit, Joaquín",664023,276542,NaN,ball,NaN,NaN,NaN,NaN,13.0,Ian Happ doubles (16) on a fly ball to left fi...,R,L,R,PIT,CHC,B,NaN,NaN,1,1,2017,-1.59,0.70,-0.88,0.48,NaN,NaN,NaN,2,8,Top,NaN,NaN,NaN,NaN,553869,NaN,NaN,7.807054,-123.288927,-7.871553,-17.665318,23.292138,-23.765604,3.74,1.66,NaN,NaN,NaN,84.8,1571.0,6.2,492193,276542,553869,605137,621559,446481,474568,607471,457705,656669,54.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,3,Changeup,1,8,8,1,8,1,8,1,Standard,Standard,250.0,0.000,0.020
4,SI,2017-09-07,94.6,-2.73,6.25,"Benoit, Joaquín",664023,276542,NaN,ball,NaN,NaN,NaN,NaN,11.0,Ian Happ doubles (16) on a fly ball to left fi...,R,L,R,PIT,CHC,B,NaN,NaN,0,1,2017,-1.27,1.19,-1.55,3.17,NaN,NaN,NaN,2,8,Top,NaN,NaN,NaN,NaN,553869,NaN,NaN,5.766298,-137.181622,-4.858858,-17.101673,30.527648,-16.579887,3.60,1.61,NaN,NaN,NaN,93.6,2149.0,5.9,492193,276542,553869,605137,621559,446481,474568,607471,457705,656669,54.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,2,Sinker,1,8,8,1,8,1,8,1,Standard,Standard,230.0,0.000,0.014


In [247]:
all_2017_stats_df.to_csv('all_2017_stats_df.csv')

In [248]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2016-04-03', '2016-10-02', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2016_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2016_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\2674255085.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2016_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SI,2016-09-27,91.5,-2.44,6.38,"Nathan, Joe",571448,150274,strikeout,swinging_strike,NaN,NaN,NaN,NaN,13.0,Nolan Arenado strikes out swinging.,R,R,R,SF,COL,S,2.0,NaN,2,2,2016,-1.41,0.59,-0.67,1.26,NaN,642162.0,518934.0,2,8,Top,NaN,NaN,NaN,NaN,457763,NaN,160927_214244,8.000700,-133.798429,-7.378708,-18.031,29.546,-23.736,3.43,1.59,NaN,NaN,NaN,NaN,NaN,NaN,449217,150274,457763,474832,605412,543216,543063,434636,452655,452254,54.5,NaN,NaN,0.0,1.0,0.0,0.0,NaN,67,5,Sinker,7,1,1,7,1,7,1,7,NaN,NaN,NaN,0.004,-0.328
1,CU,2016-09-27,80.6,-2.11,6.73,"Nathan, Joe",571448,150274,NaN,swinging_strike_blocked,NaN,NaN,NaN,NaN,14.0,Nolan Arenado strikes out swinging.,R,R,R,SF,COL,S,NaN,NaN,2,1,2016,0.27,-0.67,0.88,0.12,NaN,642162.0,518934.0,2,8,Top,NaN,NaN,NaN,NaN,457763,NaN,160927_214159,6.025317,-117.894821,-5.264160,1.297,21.976,-37.222,3.43,1.59,NaN,NaN,NaN,NaN,NaN,NaN,449217,150274,457763,474832,605412,543216,543063,434636,452655,452254,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,4,Curveball,7,1,1,7,1,7,1,7,NaN,NaN,NaN,0.000,-0.126
2,FF,2016-09-27,91.5,-1.98,6.80,"Nathan, Joe",571448,150274,NaN,ball,NaN,NaN,NaN,NaN,12.0,Nolan Arenado strikes out swinging.,R,R,R,SF,COL,B,NaN,NaN,1,1,2016,-0.76,1.46,1.21,3.79,NaN,642162.0,518934.0,2,8,Top,NaN,NaN,NaN,NaN,457763,NaN,160927_214132,9.976650,-133.852396,-4.307363,-10.986,28.051,-14.246,3.58,1.72,NaN,NaN,NaN,NaN,NaN,NaN,449217,150274,457763,474832,605412,543216,543063,434636,452655,452254,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,3,4-Seam Fastball,7,1,1,7,1,7,1,7,NaN,NaN,NaN,0.000,0.073
3,FF,2016-09-27,91.4,-2.10,6.78,"Nathan, Joe",571448,150274,NaN,foul,NaN,NaN,NaN,NaN,5.0,Nolan Arenado strikes out swinging.,R,R,R,SF,COL,S,NaN,NaN,1,0,2016,-0.65,1.45,0.17,2.75,NaN,642162.0,518934.0,2,8,Top,NaN,NaN,NaN,NaN,457763,NaN,160927_214032,7.379398,-133.666364,-6.825156,-9.099,27.598,-13.937,3.43,1.59,NaN,NaN,NaN,NaN,NaN,NaN,449217,150274,457763,474832,605412,543216,543063,434636,452655,452254,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,2,4-Seam Fastball,7,1,1,7,1,7,1,7,NaN,NaN,NaN,0.000,-0.084
4,CU,2016-09-27,79.3,-2.02,6.99,"Nathan, Joe",571448,150274,NaN,ball,NaN,NaN,NaN,NaN,14.0,Nolan Arenado strikes out swinging.,R,R,R,SF,COL,B,NaN,NaN,0,0,2016,0.32,-0.90,1.22,1.61,NaN,642162.0,518934.0,2,8,Top,NaN,NaN,NaN,NaN,457763,NaN,160927_214008,6.382383,-116.053425,-1.735432,1.609,21.647,-39.715,3.73,1.69,NaN,NaN,NaN,NaN,NaN,NaN,449217,150274,457763,474832,605412,543216,543063,434636,452655,452254,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,1,Curveball,7,1,1,7,1,7,1,7,NaN,NaN,NaN,0.000,0.039


In [249]:
all_2016_stats_df.to_csv('all_2016_stats_df.csv')

In [250]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2015-04-05', '2015-10-04', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2015_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2015_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\2494865099.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2015_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2015-10-03,92.8,-1.30,6.52,"Hawkins, LaTroy",542921,115629,field_out,hit_into_play,NaN,NaN,NaN,NaN,1.0,Tim Beckham lines out softly to second baseman...,R,R,R,TB,TOR,X,4.0,line_drive,1,1,2015,-0.60,1.45,-0.78,3.26,NaN,NaN,NaN,2,7,Bot,145.44,154.54,NaN,NaN,425900,NaN,151003_201945,2.910646,-136.242708,-5.157683,-7.86,32.65,-13.64,3.54,1.56,156.0,58.0,23.0,92.2,2101.0,6.0,416063,115629,425900,475253,446381,518626,572365,519184,607680,430832,54.5,0.36,0.324,0.0,1.0,0.0,0.0,1.0,55,3,4-Seam Fastball,2,3,2,3,3,2,2,3,Standard,Standard,NaN,-0.023,-0.092
1,FF,2015-10-03,92.2,-1.05,6.71,"Hawkins, LaTroy",542921,115629,NaN,ball,NaN,NaN,NaN,NaN,11.0,Tim Beckham lines out softly to second baseman...,R,R,R,TB,TOR,B,NaN,NaN,0,1,2015,-0.75,1.67,-1.23,4.21,NaN,NaN,NaN,2,7,Bot,NaN,NaN,NaN,NaN,425900,NaN,151003_201921,1.457997,-135.360085,-3.692584,-9.23,30.27,-11.61,3.54,1.56,NaN,NaN,NaN,91.8,2081.0,5.8,416063,115629,425900,475253,446381,518626,572365,519184,607680,430832,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,2,4-Seam Fastball,2,3,2,3,3,2,2,3,Standard,Standard,NaN,0.000,0.010
2,SL,2015-10-03,86.8,-1.79,6.49,"Hawkins, LaTroy",542921,115629,NaN,called_strike,NaN,NaN,NaN,NaN,2.0,Tim Beckham lines out softly to second baseman...,R,R,R,TB,TOR,S,NaN,NaN,0,0,2015,0.51,0.56,-0.14,3.34,NaN,NaN,NaN,2,7,Bot,NaN,NaN,NaN,NaN,425900,NaN,151003_201901,2.750832,-127.352018,-1.547203,4.77,26.28,-26.02,3.54,1.56,NaN,NaN,NaN,86.0,2271.0,5.6,416063,115629,425900,475253,446381,518626,572365,519184,607680,430832,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,1,Slider,2,3,2,3,3,2,2,3,Standard,Standard,NaN,0.000,-0.016
3,SL,2015-09-30,86.0,-1.75,6.31,"Hawkins, LaTroy",475247,115629,strikeout,foul_tip,NaN,NaN,NaN,NaN,3.0,Ryan Flaherty strikes out on a foul tip.,R,L,R,BAL,TOR,S,2.0,NaN,3,2,2015,0.12,0.54,0.30,3.69,502182.0,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,431145,NaN,150930_192802,4.593156,-125.962437,-0.169906,0.47,22.39,-26.51,3.72,1.73,NaN,NaN,NaN,NaN,NaN,NaN,416021,115629,431145,475253,446381,518626,572365,519184,607680,430832,54.5,NaN,NaN,0.0,1.0,0.0,0.0,NaN,90,7,Slider,2,15,2,15,15,2,2,15,NaN,NaN,NaN,0.000,-0.348
4,FF,2015-09-30,94.1,-1.74,6.29,"Hawkins, LaTroy",475247,115629,NaN,foul,NaN,NaN,NaN,NaN,14.0,Ryan Flaherty strikes out on a foul tip.,R,L,R,BAL,TOR,S,NaN,NaN,3,2,2015,-1.00,0.85,0.21,1.51,502182.0,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,431145,NaN,150930_192729,7.644291,-137.319084,-7.824999,-13.82,25.83,-20.23,3.72,1.73,NaN,NaN,NaN,94.2,2143.0,6.0,416021,115629,431145,475253,446381,518626,572365,519184,607680,430832,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90,6,4-Seam Fastball,2,15,2,15,15,2,2,15,Infield shift,Standard,NaN,0.000,0.000


In [251]:
all_2015_stats_df.to_csv('all_2015_stats_df.csv')

In [252]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2014-03-22', '2014-09-28', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2014_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2014_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\1634472291.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2014_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2014-09-27,93.7,-1.62,6.28,"Hawkins, LaTroy",446381,115629,field_out,hit_into_play,NaN,NaN,NaN,NaN,2.0,Darwin Barney flies out to right fielder Kyle ...,R,R,R,LAD,COL,X,9.0,fly_ball,2,2,2014,-1.05,1.42,0.20,2.78,NaN,NaN,NaN,2,8,Bot,167.67,75.3,NaN,NaN,502374,NaN,140927_210942,7.404227,-137.193847,-5.842926,-14.522,33.125,-13.576,3.25,1.48,NaN,NaN,NaN,NaN,NaN,NaN,382943,115629,502374,572019,518586,501255,542436,488681,453568,592621,54.5,NaN,NaN,0.0,NaN,0.0,0.0,NaN,72,7,4-Seam Fastball,5,4,5,4,4,5,5,4,NaN,NaN,NaN,-0.007,-0.076
1,FF,2014-09-27,93.1,-1.63,6.18,"Hawkins, LaTroy",446381,115629,NaN,foul,NaN,NaN,NaN,NaN,5.0,Darwin Barney flies out to right fielder Kyle ...,R,R,R,LAD,COL,S,NaN,NaN,2,2,2014,-1.21,1.01,-0.20,2.66,NaN,NaN,NaN,2,8,Bot,NaN,NaN,NaN,NaN,502374,NaN,140927_210913,6.773917,-136.378016,-4.770105,-16.061,30.346,-18.959,3.25,1.48,NaN,NaN,NaN,NaN,NaN,NaN,382943,115629,502374,572019,518586,501255,542436,488681,453568,592621,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,6,4-Seam Fastball,5,4,5,4,4,5,5,4,NaN,NaN,NaN,0.000,0.000
2,CU,2014-09-27,77.2,-1.53,6.58,"Hawkins, LaTroy",446381,115629,NaN,ball,NaN,NaN,NaN,NaN,14.0,Darwin Barney flies out to right fielder Kyle ...,R,R,R,LAD,COL,B,NaN,NaN,1,2,2014,0.44,-0.72,1.69,0.37,NaN,NaN,NaN,2,8,Bot,NaN,NaN,NaN,NaN,502374,NaN,140927_210850,5.941635,-113.154648,-3.401167,2.464,21.609,-37.445,3.25,1.48,NaN,NaN,NaN,NaN,NaN,NaN,382943,115629,502374,572019,518586,501255,542436,488681,453568,592621,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,5,Curveball,5,4,5,4,4,5,5,4,NaN,NaN,NaN,0.000,0.015
3,SL,2014-09-27,87.1,-1.61,6.38,"Hawkins, LaTroy",446381,115629,NaN,foul,NaN,NaN,NaN,NaN,14.0,Darwin Barney flies out to right fielder Kyle ...,R,R,R,LAD,COL,S,NaN,NaN,1,2,2014,-0.06,0.97,1.15,1.86,NaN,NaN,NaN,2,8,Bot,NaN,NaN,NaN,NaN,502374,NaN,140927_210821,6.754227,-127.424787,-5.880221,-1.869,24.236,-20.821,3.25,1.48,NaN,NaN,NaN,NaN,NaN,NaN,382943,115629,502374,572019,518586,501255,542436,488681,453568,592621,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,4,Slider,5,4,5,4,4,5,5,4,NaN,NaN,NaN,0.000,0.000
4,FF,2014-09-27,92.3,-1.57,6.10,"Hawkins, LaTroy",446381,115629,NaN,called_strike,NaN,NaN,NaN,NaN,8.0,Darwin Barney flies out to right fielder Kyle ...,R,R,R,LAD,COL,S,NaN,NaN,1,1,2014,-1.15,0.83,0.11,1.72,NaN,NaN,NaN,2,8,Bot,NaN,NaN,NaN,NaN,502374,NaN,140927_210803,7.210528,-135.074635,-6.330797,-15.178,29.835,-20.970,3.25,1.48,NaN,NaN,NaN,NaN,NaN,NaN,382943,115629,502374,572019,518586,501255,542436,488681,453568,592621,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72,3,4-Seam Fastball,5,4,5,4,4,5,5,4,NaN,NaN,NaN,0.000,-0.018


In [253]:
all_2014_stats_df.to_csv('all_2014_stats_df.csv')

In [256]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2013-03-31', '2013-09-30', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2013_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2013_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\590257304.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2013_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SL,2013-09-28,87.5,-1.88,6.24,"Hawkins, LaTroy",493114,115629,field_out,hit_into_play,NaN,NaN,NaN,NaN,6.0,"Norichika Aoki grounds out, shortstop Omar Qui...",R,L,R,NYM,MIL,X,6.0,ground_ball,2,2,2013,0.04,0.71,0.63,2.10,NaN,NaN,NaN,2,9,Top,111.45,160.64,NaN,NaN,489232,NaN,130928_191731,5.952918,-128.111131,-4.445319,-0.707,24.405,-23.768,3.08,1.43,NaN,NaN,NaN,NaN,NaN,NaN,349150,115629,489232,446263,502517,431151,435560,458913,501571,488689,54.5,NaN,NaN,0.0,NaN,0.0,0.0,NaN,70,5,Slider,1,2,2,1,2,1,2,1,NaN,NaN,NaN,0.008,-0.080
1,FF,2013-09-28,93.7,-1.69,6.17,"Hawkins, LaTroy",493114,115629,NaN,called_strike,NaN,NaN,NaN,NaN,2.0,"Norichika Aoki grounds out, shortstop Omar Qui...",R,L,R,NYM,MIL,S,NaN,NaN,2,1,2013,-1.11,1.06,0.12,2.92,NaN,488703.0,NaN,1,9,Top,NaN,NaN,NaN,NaN,489232,NaN,130928_191648,7.544815,-137.004264,-4.337540,-15.136,27.712,-18.281,3.16,1.42,NaN,NaN,NaN,NaN,NaN,NaN,349150,115629,489232,446263,502517,431151,435560,458913,501571,488689,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70,4,4-Seam Fastball,1,2,2,1,2,1,2,1,NaN,NaN,NaN,0.044,-0.073
2,FF,2013-09-28,93.4,-1.61,6.23,"Hawkins, LaTroy",493114,115629,NaN,foul,NaN,NaN,NaN,NaN,1.0,"Norichika Aoki grounds out, shortstop Omar Qui...",R,L,R,NYM,MIL,S,NaN,NaN,2,0,2013,-1.04,0.97,-0.58,2.83,NaN,488703.0,NaN,1,9,Top,NaN,NaN,NaN,NaN,489232,NaN,130928_191610,5.332880,-136.868102,-4.472962,-13.787,28.918,-19.399,3.08,1.43,NaN,NaN,NaN,NaN,NaN,NaN,349150,115629,489232,446263,502517,431151,435560,458913,501571,488689,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70,3,4-Seam Fastball,1,2,2,1,2,1,2,1,NaN,NaN,NaN,0.000,-0.060
3,SL,2013-09-28,88.0,-1.90,6.22,"Hawkins, LaTroy",493114,115629,NaN,ball,NaN,NaN,NaN,NaN,14.0,"Norichika Aoki grounds out, shortstop Omar Qui...",R,L,R,NYM,MIL,B,NaN,NaN,1,0,2013,0.47,0.33,1.37,1.62,NaN,488703.0,NaN,1,9,Top,NaN,NaN,NaN,NaN,489232,NaN,130928_191538,6.775720,-128.692994,-4.784047,3.885,23.319,-27.708,3.16,1.46,NaN,NaN,NaN,NaN,NaN,NaN,349150,115629,489232,446263,502517,431151,435560,458913,501571,488689,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70,2,Slider,1,2,2,1,2,1,2,1,NaN,NaN,NaN,0.000,0.050
4,FF,2013-09-28,93.3,-1.80,6.25,"Hawkins, LaTroy",493114,115629,NaN,ball,NaN,NaN,NaN,NaN,11.0,"Norichika Aoki grounds out, shortstop Omar Qui...",R,L,R,NYM,MIL,B,NaN,NaN,0,0,2013,-1.13,0.61,-1.06,2.89,NaN,NaN,488703.0,1,9,Top,NaN,NaN,NaN,NaN,489232,NaN,130928_191444,4.820789,-136.750752,-3.392456,-14.771,29.305,-24.060,3.05,1.43,NaN,NaN,NaN,NaN,NaN,NaN,349150,115629,489232,446263,502517,431151,435560,458913,501571,488689,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70,1,4-Seam Fastball,1,2,2,1,2,1,2,1,NaN,NaN,NaN,-0.015,0.037


In [257]:
all_2013_stats_df.to_csv('all_2013_stats_df.csv')

In [258]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2012-03-28', '2012-10-03', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2012_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2012_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\421109916.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2012_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2012-09-30,92.5,-2.02,6.22,"Hawkins, LaTroy",276545,115629,field_out,hit_into_play,NaN,NaN,NaN,NaN,4.0,"Michael Young grounds out, second baseman Howi...",R,R,R,TEX,LAA,X,4.0,ground_ball,3,2,2012,-0.95,1.09,-0.51,2.75,NaN,285078.0,443558.0,2,6,Bot,142.57,154.62,NaN,NaN,455104,NaN,120930_202347,6.289808,-135.508760,-4.812941,-12.660,26.855,-18.100,3.43,1.66,NaN,NaN,NaN,NaN,NaN,NaN,320089,115629,455104,444432,435062,430948,430895,150484,545361,116338,54.5,NaN,NaN,0.0,NaN,0.0,0.0,NaN,62,7,4-Seam Fastball,8,4,8,4,4,8,8,4,NaN,NaN,NaN,-0.008,-0.452
1,SL,2012-09-30,84.2,-2.23,6.31,"Hawkins, LaTroy",276545,115629,NaN,foul,NaN,NaN,NaN,NaN,3.0,"Michael Young grounds out, second baseman Howi...",R,R,R,TEX,LAA,S,NaN,NaN,3,2,2012,0.28,0.18,0.44,3.51,NaN,285078.0,443558.0,2,6,Bot,NaN,NaN,NaN,NaN,455104,NaN,120930_202304,5.559747,-123.198164,0.532282,1.945,19.713,-30.444,3.43,1.66,NaN,NaN,NaN,NaN,NaN,NaN,320089,115629,455104,444432,435062,430948,430895,150484,545361,116338,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,6,Slider,8,4,8,4,4,8,8,4,NaN,NaN,NaN,0.000,0.000
2,CU,2012-09-30,76.2,-1.94,6.59,"Hawkins, LaTroy",276545,115629,NaN,blocked_ball,NaN,NaN,NaN,NaN,14.0,"Michael Young grounds out, second baseman Howi...",R,R,R,TEX,LAA,B,NaN,NaN,2,2,2012,0.50,-1.05,0.72,1.37,NaN,285078.0,443558.0,2,6,Bot,NaN,NaN,NaN,NaN,455104,NaN,120930_202227,4.539201,-111.775068,-0.422794,3.262,19.851,-40.570,3.46,1.64,NaN,NaN,NaN,NaN,NaN,NaN,320089,115629,455104,444432,435062,430948,430895,150484,545361,116338,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,5,Curveball,8,4,8,4,4,8,8,4,NaN,NaN,NaN,0.000,0.115
3,FF,2012-09-30,91.8,-1.97,6.56,"Hawkins, LaTroy",276545,115629,NaN,ball,NaN,NaN,NaN,NaN,12.0,"Michael Young grounds out, second baseman Howi...",R,R,R,TEX,LAA,B,NaN,NaN,1,2,2012,-1.07,1.45,0.47,3.69,NaN,285078.0,443558.0,2,6,Bot,NaN,NaN,NaN,NaN,455104,NaN,120930_202201,8.856808,-134.164685,-4.061543,-14.234,23.119,-14.283,3.42,1.65,NaN,NaN,NaN,NaN,NaN,NaN,320089,115629,455104,444432,435062,430948,430895,150484,545361,116338,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,4,4-Seam Fastball,8,4,8,4,4,8,8,4,NaN,NaN,NaN,0.000,0.066
4,CU,2012-09-30,74.7,-1.83,6.68,"Hawkins, LaTroy",276545,115629,NaN,called_strike,NaN,NaN,NaN,NaN,2.0,"Michael Young grounds out, second baseman Howi...",R,R,R,TEX,LAA,S,NaN,NaN,1,1,2012,0.74,-1.16,0.02,3.40,NaN,285078.0,443558.0,2,6,Bot,NaN,NaN,NaN,NaN,455104,NaN,120930_202138,2.280810,-109.467888,4.102313,5.411,17.962,-41.901,3.53,1.65,NaN,NaN,NaN,NaN,NaN,NaN,320089,115629,455104,444432,435062,430948,430895,150484,545361,116338,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62,3,Curveball,8,4,8,4,4,8,8,4,NaN,NaN,NaN,0.000,-0.118


In [259]:
all_2012_stats_df.to_csv('all_2012_stats_df.csv')

In [260]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2011-03-31', '2011-09-28', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2011_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2011_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\1905862087.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2011_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2011-09-27,87.0,2.22,6.16,"Rhodes, Arthur",460131,121125,walk,ball,NaN,NaN,NaN,NaN,12.0,Brian Bogusevic walks.,R,L,L,HOU,STL,B,NaN,NaN,3,2,2011,0.98,1.53,0.84,3.94,NaN,NaN,NaN,1,7,Bot,NaN,NaN,NaN,NaN,425877,NaN,110927_214351,-5.688134,-127.594702,-1.647750,11.529,27.025,-15.713,3.54,1.62,NaN,NaN,NaN,NaN,NaN,NaN,289306,121125,425877,405395,444135,501896,346857,501800,445055,204020,54.5,NaN,NaN,0.7,NaN,0.0,0.0,NaN,67,6,4-Seam Fastball,6,9,6,9,9,6,6,9,NaN,NaN,NaN,0.031,0.209
1,FF,2011-09-27,89.6,2.09,6.15,"Rhodes, Arthur",460131,121125,NaN,ball,NaN,NaN,NaN,NaN,13.0,Brian Bogusevic walks.,R,L,L,HOU,STL,B,NaN,NaN,2,2,2011,1.19,1.28,-1.56,1.59,NaN,NaN,NaN,1,7,Bot,NaN,NaN,NaN,NaN,425877,NaN,110927_214330,-11.935734,-130.702917,-7.225282,15.735,27.692,-16.458,3.54,1.70,NaN,NaN,NaN,NaN,NaN,NaN,289306,121125,425877,405395,444135,501896,346857,501800,445055,204020,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,5,4-Seam Fastball,6,9,6,9,9,6,6,9,NaN,NaN,NaN,0.000,0.067
2,FF,2011-09-27,86.8,2.11,6.21,"Rhodes, Arthur",460131,121125,NaN,swinging_strike,NaN,NaN,NaN,NaN,11.0,Brian Bogusevic walks.,R,L,L,HOU,STL,S,NaN,NaN,2,1,2011,0.83,1.53,-1.18,3.37,NaN,NaN,NaN,1,7,Bot,NaN,NaN,NaN,NaN,425877,NaN,110927_214307,-9.846678,-126.886772,-3.098437,10.612,25.057,-15.554,3.54,1.70,NaN,NaN,NaN,NaN,NaN,NaN,289306,121125,425877,405395,444135,501896,346857,501800,445055,204020,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,4,4-Seam Fastball,6,9,6,9,9,6,6,9,NaN,NaN,NaN,0.000,-0.048
3,FF,2011-09-27,87.3,2.11,6.15,"Rhodes, Arthur",460131,121125,NaN,ball,NaN,NaN,NaN,NaN,13.0,Brian Bogusevic walks.,R,L,L,HOU,STL,B,NaN,NaN,1,1,2011,0.96,1.33,-0.83,1.62,NaN,NaN,NaN,1,7,Bot,NaN,NaN,NaN,NaN,425877,NaN,110927_214249,-9.395379,-127.516176,-6.733056,12.094,26.656,-16.740,3.71,1.73,NaN,NaN,NaN,NaN,NaN,NaN,289306,121125,425877,405395,444135,501896,346857,501800,445055,204020,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,3,4-Seam Fastball,6,9,6,9,9,6,6,9,NaN,NaN,NaN,0.000,0.035
4,FF,2011-09-27,87.9,2.08,6.22,"Rhodes, Arthur",460131,121125,NaN,ball,NaN,NaN,NaN,NaN,13.0,Brian Bogusevic walks.,R,L,L,HOU,STL,B,NaN,NaN,0,1,2011,1.15,1.30,-1.11,1.68,NaN,NaN,NaN,1,7,Bot,NaN,NaN,NaN,NaN,425877,NaN,110927_214232,-10.489527,-128.331786,-6.848008,14.422,26.080,-16.879,3.67,1.76,NaN,NaN,NaN,NaN,NaN,NaN,289306,121125,425877,405395,444135,501896,346857,501800,445055,204020,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67,2,4-Seam Fastball,6,9,6,9,9,6,6,9,NaN,NaN,NaN,0.000,0.016


In [261]:
all_2011_stats_df.to_csv('all_2011_stats_df.csv')

In [262]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2010-04-04', '2010-10-03', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2010_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2010_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\118814614.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2010_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SL,2010-10-03,82.8,1.82,6.45,"Rhodes, Arthur",456715,121125,walk,ball,NaN,NaN,NaN,NaN,13.0,Lorenzo Cain walks.,R,R,L,CIN,MIL,B,NaN,NaN,3,2,2010,-0.57,-0.02,-0.74,-0.06,NaN,NaN,NaN,2,8,Top,NaN,NaN,NaN,NaN,280061,NaN,101003_152918,-4.524183,-120.941924,-7.207636,-4.792,18.730,-31.203,3.63,1.80,NaN,NaN,NaN,NaN,NaN,NaN,266228,121125,280061,111867,453362,464433,457926,425556,502317,217100,54.5,NaN,NaN,0.7,NaN,0.0,0.0,NaN,65,6,Slider,3,2,2,3,2,3,2,3,NaN,NaN,NaN,-0.033,0.101
1,SL,2010-10-03,81.1,1.76,6.46,"Rhodes, Arthur",456715,121125,NaN,ball,NaN,NaN,NaN,NaN,13.0,Lorenzo Cain walks.,R,R,L,CIN,MIL,B,NaN,NaN,2,2,2010,-0.64,-0.30,-0.95,0.72,NaN,NaN,NaN,2,8,Top,NaN,NaN,NaN,NaN,280061,NaN,101003_152857,-4.633618,-118.613714,-4.330090,-5.080,21.113,-34.174,3.65,1.83,NaN,NaN,NaN,NaN,NaN,NaN,266228,121125,280061,111867,453362,464433,457926,425556,502317,217100,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,5,Slider,3,2,2,3,2,3,2,3,NaN,NaN,NaN,0.000,0.038
2,FF,2010-10-03,90.3,1.94,6.31,"Rhodes, Arthur",456715,121125,NaN,ball,NaN,NaN,NaN,NaN,13.0,Lorenzo Cain walks.,R,R,L,CIN,MIL,B,NaN,NaN,1,2,2010,0.43,1.54,-1.84,1.57,NaN,NaN,NaN,2,8,Top,NaN,NaN,NaN,NaN,280061,NaN,101003_152838,-10.432844,-131.690814,-8.518459,6.879,25.487,-13.112,3.69,1.80,NaN,NaN,NaN,NaN,NaN,NaN,266228,121125,280061,111867,453362,464433,457926,425556,502317,217100,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,4,4-Seam Fastball,3,2,2,3,2,3,2,3,NaN,NaN,NaN,0.000,0.018
3,SL,2010-10-03,80.4,1.71,6.55,"Rhodes, Arthur",456715,121125,NaN,ball,NaN,NaN,NaN,NaN,13.0,Lorenzo Cain walks.,R,R,L,CIN,MIL,B,NaN,NaN,0,2,2010,-0.51,-0.02,-1.04,1.90,NaN,NaN,NaN,2,8,Top,NaN,NaN,NaN,NaN,280061,NaN,101003_152819,-4.970231,-117.732494,-2.309743,-3.617,22.976,-31.884,3.72,1.80,NaN,NaN,NaN,NaN,NaN,NaN,266228,121125,280061,111867,453362,464433,457926,425556,502317,217100,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,3,Slider,3,2,2,3,2,3,2,3,NaN,NaN,NaN,0.000,0.010
4,SL,2010-10-03,80.7,1.76,6.52,"Rhodes, Arthur",456715,121125,NaN,swinging_strike,NaN,NaN,NaN,NaN,13.0,Lorenzo Cain walks.,R,R,L,CIN,MIL,S,NaN,NaN,0,1,2010,-0.61,-0.09,-0.52,1.59,NaN,NaN,NaN,2,8,Top,NaN,NaN,NaN,NaN,280061,NaN,101003_152801,-3.735821,-118.137504,-2.970547,-4.951,19.275,-32.490,3.57,1.76,NaN,NaN,NaN,NaN,NaN,NaN,266228,121125,280061,111867,453362,464433,457926,425556,502317,217100,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,2,Slider,3,2,2,3,2,3,2,3,NaN,NaN,NaN,0.000,-0.022


In [263]:
all_2010_stats_df.to_csv('all_2010_stats_df.csv')

In [264]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2009-04-05', '2009-10-06', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2009_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2009_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\1425485374.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2009_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,SL,2009-09-22,80.1,1.67,6.67,"Rhodes, Arthur",425491,121125,strikeout,swinging_strike_blocked,NaN,NaN,NaN,NaN,13.0,"Ryan Doumit strikes out swinging, catcher Ramo...",R,R,L,PIT,CIN,S,2.0,NaN,1,2,2009,-0.52,-0.70,-0.84,0.09,NaN,NaN,NaN,2,7,Bot,NaN,NaN,NaN,NaN,150421,NaN,090922_215003,-4.389443,-117.087026,-5.082563,-3.957,19.039,-37.673,3.43,1.52,NaN,NaN,NaN,NaN,NaN,NaN,246447,121125,150421,458015,408252,121409,457926,425556,453211,457803,54.5,NaN,NaN,0.0,NaN,0.0,0.0,NaN,68,4,Slider,3,8,3,8,8,3,3,8,NaN,NaN,NaN,-0.003,-0.073
1,SL,2009-09-22,80.7,1.57,6.62,"Rhodes, Arthur",425491,121125,NaN,swinging_strike,NaN,NaN,NaN,NaN,13.0,"Ryan Doumit strikes out swinging, catcher Ramo...",R,R,L,PIT,CIN,S,NaN,NaN,1,1,2009,-0.56,-0.41,-1.04,0.80,NaN,NaN,NaN,2,7,Bot,NaN,NaN,NaN,NaN,150421,NaN,090922_214936,-4.559784,-118.109257,-4.151201,-4.269,21.402,-35.174,3.43,1.52,NaN,NaN,NaN,NaN,NaN,NaN,246447,121125,150421,458015,408252,121409,457926,425556,453211,457803,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,3,Slider,3,8,3,8,8,3,3,8,NaN,NaN,NaN,0.000,-0.028
2,FF,2009-09-22,90.9,1.71,6.57,"Rhodes, Arthur",425491,121125,NaN,ball,NaN,NaN,NaN,NaN,8.0,"Ryan Doumit strikes out swinging, catcher Ramo...",R,R,L,PIT,CIN,B,NaN,NaN,0,1,2009,0.88,1.38,-0.26,2.16,NaN,NaN,NaN,2,7,Bot,NaN,NaN,NaN,NaN,150421,NaN,090922_214919,-7.138286,-132.926061,-7.508049,11.578,26.497,-14.777,3.37,1.67,NaN,NaN,NaN,NaN,NaN,NaN,246447,121125,150421,458015,408252,121409,457926,425556,453211,457803,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,2,4-Seam Fastball,3,8,3,8,8,3,3,8,NaN,NaN,NaN,0.000,0.013
3,SL,2009-09-22,80.7,1.73,6.67,"Rhodes, Arthur",425491,121125,NaN,called_strike,NaN,NaN,NaN,NaN,5.0,"Ryan Doumit strikes out swinging, catcher Ramo...",R,R,L,PIT,CIN,S,NaN,NaN,0,0,2009,-0.51,-0.10,0.21,2.19,NaN,NaN,NaN,2,7,Bot,NaN,NaN,NaN,NaN,150421,NaN,090922_214901,-2.243166,-118.148051,-1.890828,-4.287,20.830,-32.739,3.41,1.58,NaN,NaN,NaN,NaN,NaN,NaN,246447,121125,150421,458015,408252,121409,457926,425556,453211,457803,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,1,Slider,3,8,3,8,8,3,3,8,NaN,NaN,NaN,0.000,-0.018
4,SL,2009-09-22,82.5,1.60,6.67,"Rhodes, Arthur",434540,121125,field_out,hit_into_play,NaN,NaN,NaN,NaN,13.0,Garrett Jones flies out to left fielder Laynce...,R,L,L,PIT,CIN,X,7.0,fly_ball,2,2,2009,-0.68,0.05,-0.94,2.30,NaN,NaN,NaN,1,7,Bot,72.29,122.49,NaN,NaN,150421,NaN,090922_214825,-4.248895,-120.829471,-2.382552,-5.676,21.849,-31.268,3.72,1.81,NaN,NaN,NaN,NaN,NaN,NaN,246447,121125,150421,458015,408252,121409,457926,425556,453211,457803,54.5,NaN,NaN,0.0,NaN,0.0,0.0,NaN,67,5,Slider,3,8,3,8,8,3,3,8,NaN,NaN,NaN,-0.006,-0.156


In [265]:
all_2009_stats_df.to_csv('all_2009_stats_df.csv')

In [266]:
all_player_stats = []

# Loop through each MLBAM ID in the pitcher_data_df
for key_mlbam in pitcher_data_df['key_mlbam']:
    # Fetch pitching stats from statcast
    player_stats = statcast_pitcher('2008-03-25', '2008-09-30', key_mlbam)
    # Append the fetched stats to the list
    all_player_stats.append(player_stats)

# Concatenate all DataFrames into a single DataFrame
all_2008_stats_df = pd.concat(all_player_stats, ignore_index=True)

all_2008_stats_df.head()

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering Player Data
Gathering 

C:\Users\johns\AppData\Local\Temp\ipykernel_16196\2286481764.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_2008_stats_df = pd.concat(all_player_stats, ignore_index=True)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
0,FF,2008-09-28,94.4,2.17,6.08,"Rhodes, Arthur",113232,121125,field_out,hit_into_play,NaN,NaN,NaN,NaN,5.0,Carlos Delgado lines out to left fielder Josh ...,R,L,L,NYM,MIA,X,7.0,line_drive,0,1,2008,1.03,1.41,-0.15,2.40,NaN,408314.0,136860.0,2,8,Bot,69.28,72.29,NaN,NaN,434633,NaN,080928_163635,-8.707990,-137.912339,-6.509925,14.542,26.437,-13.292,3.23,1.67,NaN,NaN,NaN,NaN,NaN,NaN,236178,121125,434633,408305,462564,136500,407792,425545,457727,425496,54.5,NaN,NaN,0.0,NaN,0.0,0.0,NaN,65,2,4-Seam Fastball,2,4,2,4,4,2,2,4,NaN,NaN,NaN,-0.082,-0.379
1,FF,2008-09-28,93.0,2.21,6.03,"Rhodes, Arthur",113232,121125,NaN,swinging_strike,NaN,NaN,NaN,NaN,12.0,Carlos Delgado lines out to left fielder Josh ...,R,L,L,NYM,MIA,S,NaN,NaN,0,0,2008,0.83,1.36,1.02,3.12,NaN,408314.0,136860.0,2,8,Bot,NaN,NaN,NaN,NaN,434633,NaN,080928_163609,-5.173211,-136.112715,-4.206386,11.045,23.637,-14.797,3.23,1.67,NaN,NaN,NaN,NaN,NaN,NaN,236178,121125,434633,408305,462564,136500,407792,425545,457727,425496,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,1,4-Seam Fastball,2,4,2,4,4,2,2,4,NaN,NaN,NaN,0.000,-0.072
2,NaN,2008-09-26,NaN,NaN,NaN,"Rhodes, Arthur",113232,121125,field_out,hit_into_play,NaN,NaN,NaN,NaN,NaN,"Carlos Delgado grounds out, second baseman Dan...",R,L,L,NYM,MIA,X,4.0,ground_ball,1,0,2008,NaN,NaN,NaN,NaN,NaN,NaN,431151.0,2,7,Bot,157.63,141.57,NaN,NaN,434633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236148,121125,434633,408305,462564,136500,434670,425545,407792,425496,NaN,NaN,NaN,0.0,NaN,0.0,0.0,NaN,65,2,NaN,1,5,1,5,5,1,1,5,NaN,NaN,NaN,-0.013,-0.264
3,NaN,2008-09-26,NaN,NaN,NaN,"Rhodes, Arthur",113232,121125,NaN,ball,NaN,NaN,NaN,NaN,NaN,"Carlos Delgado grounds out, second baseman Dan...",R,L,L,NYM,MIA,B,NaN,NaN,0,0,2008,NaN,NaN,NaN,NaN,NaN,NaN,431151.0,2,7,Bot,NaN,NaN,NaN,NaN,434633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236148,121125,434633,408305,462564,136500,434670,425545,407792,425496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,1,NaN,1,5,1,5,5,1,1,5,NaN,NaN,NaN,0.000,0.032
4,FF,2008-09-22,92.2,1.87,6.72,"Rhodes, Arthur",279913,121125,strikeout,swinging_strike,NaN,NaN,NaN,NaN,1.0,Corey Patterson strikes out swinging.,R,L,L,CIN,MIA,S,2.0,NaN,2,2,2008,0.97,1.37,-0.51,2.89,433898.0,NaN,425429.0,2,7,Bot,NaN,NaN,NaN,NaN,276354,NaN,080922_153603,-8.500073,-134.717333,-6.349345,13.193,25.168,-14.613,3.43,1.61,NaN,NaN,NaN,NaN,NaN,NaN,234376,121125,276354,408312,462564,136500,434670,425545,457727,435181,54.5,NaN,NaN,0.0,NaN,0.0,0.0,NaN,65,5,4-Seam Fastball,7,4,7,4,4,7,7,4,NaN,NaN,NaN,-0.011,-0.372


In [267]:
all_2008_stats_df.to_csv('all_2008_stats_df.csv')

In [ ]:
all_2019_stats_df.to_csv('all_2019_stats_df.csv')

In [ ]:
all_2020_stats_df.to_csv('all_2020_stats_df.csv')

In [ ]:
all_2021_stats_df.to_csv('all_2021_stats_df.csv')

In [ ]:
all_2022_stats_df.to_csv('all_2022_stats_df.csv')

In [ ]:
all_2023_stats_df.info()

In [ ]:
all_2023_stats_df.drop(columns=['batter', 'events', 'description', 'zone',
                                'des', 'game_type', 'stand', 'home_team',
                                'away_team', 'type', 'hit_location', 'bb_type',
                                'balls', 'strikes', 'pfx_x',
                                'pfx_z', 'plate_x', 'plate_z', 'on_3b',
                                'on_2b', 'on_1b', 'outs_when_up', 'inning',
                                'inning_topbot', 'hc_x', 'hc_y', 'fielder_2',
                                'umpire', 'sv_id', 'hit_distance_sc', 'sz_top',
                                 'sz_bot', 'launch_speed', 'launch_angle', 'game_pk',
                                'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4',
                                'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8',
                                'fielder_9', 'estimated_ba_using_speedangle',
                                'estimated_woba_using_speedangle', 'babip_value',
                                'launch_speed_angle', 'woba_value', 'woba_denom', 'iso_value', 
                                'at_bat_number', 'pitch_number', 'home_score', 'away_score',
                                'bat_score', 'fld_score', 'post_home_score', 'post_bat_score',
                                'post_fld_score', 'post_away_score', 'if_fielding_alignment',
                                'of_fielding_alignment', 'delta_home_win_exp',
                                'delta_run_exp', 'spin_rate_deprecated', 'break_angle_deprecated',
                               'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated'], inplace=True)
all_2023_stats_df

In [ ]:
all_2023_stats_df.to_csv('all_2023_stats_df.csv')

In [ ]:
all_2023_stats_df

May need to completely remove 'depreciated' columns...

Check for NaN values, fill w/ mean for most columns

In [ ]:
# Group by 'game_date' and 'pitcher' to calculate the total pitches
total_pitches = all_2023_stats_df.groupby(['game_date', 'pitcher']).size().reset_index(name='total_pitches')

# Group by 'game_date', 'pitcher', and 'pitch_type' to calculate the sum total of each pitch_type
total_pitches_by_type = all_2023_stats_df.groupby(['game_date', 'pitcher', 'pitch_type']).size().reset_index(name='count_by_pitch_type')

# Calculate averages of the specified metrics for each pitch type, grouped by game_date and pitcher
avg_metrics = all_2023_stats_df.groupby(['game_date', 'pitcher', 'pitch_type']).agg({
    'release_speed': 'mean',
    'release_pos_x': 'mean',
    'release_pos_z': 'mean',
    'spin_dir': 'mean',
    'vx0': 'mean',
    'vy0': 'mean',
    'vz0': 'mean',
    'ax': 'mean',
    'ay': 'mean',
    'az': 'mean',
    'effective_speed': 'mean',
    'release_spin_rate': 'mean',
    'release_extension': 'mean',
    'release_pos_y': 'mean',
    'spin_axis': 'mean'
}).reset_index()

# Merging total pitches and total pitches by type back if needed
# This step depends on how you want to structure your final DataFrame. You can merge these summary DataFrames
# based on 'game_date' and 'pitcher' columns to get a comprehensive summary DataFrame.

# Example merge (you can adjust or expand based on what you exactly need):
grouped_2023_df = total_pitches.merge(total_pitches_by_type, on=['game_date', 'pitcher'], how='left')
grouped_2023_df = final_df.merge(avg_metrics, on=['game_date', 'pitcher', 'pitch_type'], how='left')

# Note: This example assumes you want to merge all these statistics back together. You might adjust the merging
# strategy based on your analysis needs, including how you want to handle the multi-level grouping and aggregation.


In [ ]:
final_df

In [ ]:
final_df['pitcher'].value_counts()

Use tj df or final df from other notebook?

In [ ]:
surgery_df = hist_tj_df[hist_tj_df['Surgery'] != 0]
surgery_df

Need to clean this part up and streamline. Too disorganized.

In [ ]:
# Correct approach to identify missing 'key_mlbam' values
missing_key_mlbam_df = surgery_df[surgery_df['key_mlbam'].isna()][['Name', 'key_mlbam']]
missing_key_mlbam_df

In [ ]:
updates = {
    'burnett a. j.': '150359',
    'wilson c. j.': '450351',
    'griffin a. j. ': '456167'
}

for name, key_mlbam in updates.items():
    surgery_df.loc[surgery_df['Name'] == name, 'key_mlbam'] = key_mlbam

In [ ]:
# Manual input for the missing 'key_mlbam' values based on the best matched names

# The key_mlbam values were obtained previously for the updated names
key_mlbam_updates = {
    "José Mesa": 118294,
    "Francisco Córdova": 150207,
    "Hipólito Pichardo": 120632,
    "Osvaldo Fernández": 136880,
    "José Rijo": 121250,
    "Geremi González": 150282,
    "Runelvys Hernández": 407908,
    "Odalis Pérez": 150229,
    "José Contreras": 150215,
    "Rodrigo López": 408061,
    "Jo-Jo Reyes": 457447,
    "Érik Bédard": 430641,
    "A. J. Burnett": 150359,
    "Félix Doubront": 467094,
    "C. J. Wilson": 450351,
    "José Fernández": 605228,
    "Alfredo Simón": 430604,
    "A. J. Griffin": 456167,
    "Wei-Yin Chen": 612672,
    "Roenis Elías": 602904,
    "Jaime García": 448802,
    "Miguel González": 456068,
    "Héctor Noesí": 456051,
    "Iván Nova": 467100,
    "Carlos Martínez": 593372,
    "Lance McCullers Jr.": 621121,
    "Aníbal Sánchez": 434671,
    "Sandy Alcántara": 645261,
    "Matthew Boyd": 571510,
    "Domingo Germán": 593334,
    "Pablo López": 641154,
    "Jesús Luzardo": 666200,
    "Germán Márquez": 608566,
    "Martín Pérez": 527048,
    "Carlos Rodón": 607074,
    "José Urquidy": 664353,
    "Víctor Zambrano": 406913,
    "Hong-Chih Kuo": 425539,
    "Edinson Vólquez": 450172,
    "Jose Alvarez": 571439
}

# Update the 'key_mlbam' in surgery_df based on the updated names
for name, mlbam in key_mlbam_updates.items():
    surgery_df.loc[surgery_df['Name'] == name, 'key_mlbam'] = mlbam

surgery_df


Must be due to the preprocessing - removing special characters, etc.

Okay, all MLB ID's are now in.